## Лабораторная работа №5 (Поиск по векторной БД)


Необходимо записать ваш датасет в векторную базу данных и выполнить эксперименты по поиску схожих фрагментов текста, соответствующих запросу.



### Import & set up

In [ ]:
!pip uninstall llmx==0.0.15a0

In [ ]:
!pip install lida==0.0.10
!pip install llmx==0.0.15a0

In [ ]:
!pip install openai
!pip install evaluate
!pip install llama-cpp-python
!pip install pinecone-client
!pip install langchain==0.0.300
!pip install chromadb==0.4.12
!pip install sentence-transformers==2.2.2

In [ ]:
import pandas as pd
import numpy as np
import statistics
import pinecone
import glob
import os
import chromadb
import nltk
from langchain.document_loaders import PDFMinerLoader, TextLoader, CSVLoader, UnstructuredWordDocumentLoader, UnstructuredHTMLLoader
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from multiprocessing.pool import ThreadPool
from langchain.vectorstores import Chroma
from langchain.schema import Document
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from typing import Any
from tqdm import tqdm
from nltk.tokenize import sent_tokenize

## Show data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/nlp-news/train.csv', header=None, names = ['category','title', 'text'])
df['ID'] = df.index
df.head(10)

## Разбиение текстовых документов на фрагменты.

> Разработать алгоритм разбиения текстовых документов на фрагменты текста. Можно использовать уже существующие механизмы, например, разбиение по длине фрагмента текста в символах и пересечению с соседними фрагментами. Дополнительные баллы за усложненные варианты, например: учитывать границы токенов и предложений.

> Подготовка метаданных для каждого фрагмента, таких как класс документа, автор документа, ключевые слова и др.

In [ ]:
nltk.download('punkt')

def cut_text_by_sent(text, fragment_len, overlay):
    sentences = sent_tokenize(text)
    fragments = []
    current_fragment = []
    current_len = 0

    for sent in sentences:
        if current_len + len(sent) <= fragment_len:
            current_fragment.append(sent)
            current_len += len(sent)
        else:
            if current_fragment:
                fragments.append(' '.join(current_fragment))
            current_fragment = [sent]
            current_len = len(sent)

    final_fragments = []

    for fragment in fragments:
        if len(fragment) >= fragment_len:
            all_len = 0
            len_text = len(fragment)
            while all_len + fragment_len <= len_text:
                final_fragments.append(fragment[all_len:all_len + fragment_len])
                all_len += overlay
        else:
            final_fragments.append(fragment)

    return final_fragments

In [ ]:
long_text = 'Магистрант, совмещающий обучение с трудовой деятельностью, вправе проходить практику по месту трудовой деятельности, если профессиональная деятельность, осуществляемая им, соответствует требованиям к содержанию практики. Магистрант должен своевременно подать заявление (см. Приложение 2) с заверенной в организации копией трудового договора и согласие на обработку персональных данных (см. Приложение 3) куратору практики. Куратор практики подтверждает соответствие профессиональной деятельности требованиям к практике или отклоняет место практики из-за несоответствия профессиональной деятельности требованиям к практике.'
text_fragments = cut_text_by_sent(long_text, 200, 100)

In [ ]:
print('Text sample: ', long_text)
print('Result:', text_fragments)
print('Size: ', [len(f) for f in text_fragments])

## Векторизация фрагментов текста

Векторизация фрагментов текста. В качестве метода векторизации можно использовать стороний api (huggingface, openai, etc.), w2v или любую другую модель на выбор. Рекомендуется применить модель с huggingface. Подходящие модели с huggingface можно выбрать по ссылке.

Модель: paraphrase-multilingual-mpnet-base-v2

Создание Векторной Базы Данных (ВБД).

Необходимо выбрать одну из доступных ВБД, например: Chroma (рекомендуемая с точки зрения простоты), Pinecone и т.д.

Реализовать механизм загрузки и сохранения текстовых данных в ВБД.

In [ ]:
class Loader:

  def load_single_document(self, file_path: str):
    pass

  def load_documents(self, source_dir: str):
    pass

class Embedder():
  # vectorization via SentenceTransformer
  def __init__(self):
    self.model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

  # vect for sentense
  def get_embeddings(self, sentences):
    return [[float(e) for e in list(emb)] for emb in list(self.model.encode(sentences))]

class ChromaDB():
  # ChromaDB Client Init
  def __init__(self):

    # self.client = chromadb.Client()
    # Disk connect
    self.client = chromadb.PersistentClient(path="/content/gdrive/MyDrive/nlp-news")

  def clear(self, name):
    self.client.delete_collection(name=name)
    return self.client.list_collections()

  def get_collection(self, name):
    return self.client.get_collection(name=name)

  def get_collections(self):
    return self.client.list_collections()

class ChromaCollection():
  def __init__(self, collection_name, similarity, client):
    self.collection_name = collection_name
    self.similarity = similarity
    self.client = client
    self.collection = self.client.get_or_create_collection(name=collection_name, metadata={"hnsw:space": similarity})

  def add(self, embeddings, texts, topics, ids):
    self.collection.add(
        embeddings = embeddings,
         documents = texts,
         metadatas = [{"source": "df", "category":f"{topic}"} for i, topic in enumerate(topics)],
         ids = [f'id {i}' for i in ids]
)

  # get number of results simular data via embeddings
  def query(self, embeddings, n_results):
    return self.collection.query(
      query_embeddings=embeddings,
       n_results=n_results,
    )

  # get all doc
  def get(self):
    return self.collection.get()

  # count doc in collection
  def count(self):
    return self.collection.count()

In [ ]:
embedder = Embedder()

In [ ]:
embeds = embedder.get_embeddings(df['text'][:30000])


In [ ]:
client = ChromaDB()
client.get_collections()

## Поиск схожих фрагментов текста
Выбрать алгоритмы similarity для поиска схожих фрагментов текста.
Реализовать механизм поиска схожих фрагментов по заданным запросам.

* **Косинусное сходство (Cosine Similarity)**: Этот алгоритм измеряет косинус угла между двумя векторами, представляющими текстовые фрагменты. Более высокое значение косинусного сходства указывает на более близкое сходство между фрагментами.
* **Евклидово расстояние (Euclidean Distance)**: Этот алгоритм измеряет расстояние между двумя точками в n-мерном пространстве. Для текстовых фрагментов, которые представлены как точки в пространстве, меньшее значение евклидова расстояния указывает на более близкое сходство.
* **IP-расстояние (Integral Projection Distance)**: Этот алгоритм измеряет сходство между двумя распределениями, основываясь на их форме и значении проекций. Для этого оно вычисляет площадь между интегральными проекциями двух распределений. Чем меньше площадь между проекциями, тем больше схожесть между распределениями.

In [ ]:
# 'cos_sim', 'l2_sim' и 'Ip_sim' - имена коллекций, которые будут созданы в БД ChromaDB
# 'cosine', 'l2' и 'ip' - типы схожести (similarity) для каждой коллекции.
# cosine - косинусное расстояние
# l2 - евклидово расстояние
# ip - произведение скалярного умножения
collection_cos = ChromaCollection('cos_sim', 'cosine', client.client)
collection_l2 = ChromaCollection('l2_sim', 'l2', client.client)
collection_Ip = ChromaCollection('Ip_sim', 'ip', client.client)

In [ ]:
# embeds - векторные представления документов, которые нужно добавить в коллекцию
# texts - тексты документов, которые нужно добавить в коллекцию
# topics - темы (метаданные) документов, которые нужно добавить в коллекцию
# ids - идентификаторы документов, которые нужно добавить в коллекцию
collection_cos.add(embeds[0:30000], df['text'].values.tolist()[0:30000], df['category'].values.tolist()[0:30000], df['ID'].values.tolist()[0:30000])
collection_l2.add(embeds[0:30000], df['text'].values.tolist()[0:30000], df['category'].values.tolist()[0:30000], df['ID'].values.tolist()[0:30000])
collection_Ip.add(embeds[0:30000], df['text'].values.tolist()[0:30000], df['category'].values.tolist()[0:30000], df['ID'].values.tolist()[0:30000])

In [ ]:
#num of samples
print(collection_cos.count())
print(collection_l2.count())
print(collection_Ip.count())

## Оценка качества поиска

Сгенерировать набор запросов к ВБД.

Провести оценку качества поиска, определяя, насколько хорошо схожие фрагменты отображаются в результатах поиска. Оценку можно выполнить следующими способами:
на основе ручной оценки качества запросов и соответствующих ответов;
посчитать средний порядковый номер требуемого фрагмента в отсортированном по релевантности спике результатов.

In [ ]:
# Test samples
news = [
    "Unknown Nesterenko Makes World Headlines (Reuters). Reuters - Belarus Yuliya Nesterenko won the top\women's athletics gold medal at the Olympics on Saturday,\triumphing over a field stripped of many big names because of\doping woes to win the 100 meters.",
    "Producer sues for Rings profits Hollywood producer Saul Zaentz sues the producers of The Lord of the Rings for \$20m in royalties.",
    "South Korean police used water cannon in central Seoul Sunday to disperse at least 7,000 protesters urging the government to reverse a controversial decision to\send more troops to Iraq.",
    "Russia ready to contribute to settlement of South Ossetia conflict: Putin. MOSCOW, Aug. 18 (Xinhuanet) -- Russian President Vladimir Putin said Wednesday that Russia is ready to contribute to a settlement of conflict between Georgia and its separatist province of South Ossetia.",
    "Hobbit-finding Boffins in science top 10. AP - Australian scientists who helped discover a species of tiny humans nicknamed Hobbits have been hailed for making the second most important scientific achievement of 2004",
    "Kevin Hartman made seven saves for Los Angeles, and Jon Busch had two saves for Columbus as the Galaxy and Crew played to a 0-0 tie Saturday night.",
    "Exploring Andromeda (SPACE.com) SPACE.com - Although winter officially begins on Dec. 21 at 7:40 a.m. EST, \  one of the landmarks of the autumn sky is still readily visible, high toward \  the south around 7 p.m. local time.",
    "Pricey Drug Trials Turn Up Few New Blockbusters The \$500 billion drug industry is stumbling badly in its core business of finding new medicines, while aggressively marketing existing drugs.",
    "PRESS START FOR NOSTALGIA Like Led Zeppelin #39;s  #39; #39;Stairway to Heaven #39; #39; and Lynyrd Skynyrd #39;s  #39; #39;Freebird, #39; #39; classic video games like Frogger and Pong can bring back an entire era.",
    "Russia shrugs off US court freeze on oil giant Yukos auction MOSCOW (AFP) - Russia forged ahead with the weekend auction of the core asset of crippled oil giant Yukos despite a disputed US court order barring the sale, with state-controlled gas giant Gazprom entering the bidding.",
    "NASA #39;s departing chief, Sean O #39;Keefe, on Friday defended his decision to pursue a robotic repair mission to the Hubble Space Telescope, days after a panel of scientists said a shuttle mission would be better.",
    "Cisco invests \$12 million in Japan R amp;D center On Thursday, the company announced it will invest \$12 million over the next five years in a new research and development center in Tokyo.",
    "Michael Phelps took care of qualifying for the Olympic 200-meter freestyle semifinals Sunday, and then found out he had been added to the American team for the evening's 400 freestyle relay final. Phelps' rivals Ian Thorpe and Pieter van den Hoogenband and teammate Klete Keller were faster than the teenager in the 200 free preliminaries.",
    "Whitman: EBay To Buy Rent.com; Compliments Craigslist Stake Without reserve. EBay (nasdaq: EBAY - news - people ) on Friday said it is buying Rent.com. The latter, which is privately held, provides online listings of apartment and house rentals",
    "Sporadic gunfire and shelling took place overnight in the disputed Georgian region of South Ossetia in violation of a fragile ceasefire, wounding seven Georgian servicemen.",
    "Robinho #39;s mother releaased by kidnappers The mother of Santos striker Robinho was released unharmed on Friday, 40 days after she was kidnapped at a family gathering. Marina da Silva de Souza, 44, appeared healthy but thinner than when she was abducted"
    ]

#Related questions
questions = [
    "What has Yuliya Nesterenko won?",
    "Which movie's producers are being sued by Saul Zaentz?",
    "Why did South Korean police use water cannon in central Seoul?",
    "What is Russia ready to do with South Ossetia conflict?",
    "What was the second most important scientific achievement of 2004?",
    "How did Kevin Hartman and Jon Busch influence the outcome of the match?",
    "What time and date the winter officially begins?",
    "How much billion the drug industry is lost?",
    "Which games can bring back an entire era?",
    "Who is court freeze on oil giant Yukos auction with state-controlled gas giant Gazprom entering the bidding?",
    "Who was the Departing Chief of NASA?",
    "In which city is a development center that Cisco invests 12 million?",
    "Who did take care of qualifying for the Olympic 200-meter freestyle semifinals?",
    "What is Rent.com that EBay bought?",
    "What happened in the disputed Georgian region of South Ossetia overnight, violating the fragile ceasefire and causing injuries to Georgian servicemen?",
    "How many days have past since she was kidnapped at a family gathering?"
    ]


answers = [
    "The top\women's athletics gold medal",
    "The Lord of the Rings",
    "To disperse at least 7,000 protesters urging the government to reverse a controversial decision to\send more troops to Iraq.",
    "Ready to contribute to settlement of South Ossetia conflict",
    "Discover a species of tiny humans nicknamed Hobbits",
    "They made saves for their respective teams, resulting in a 0-0 tie",
    "Winter officially begins on Dec. 21 at 7:40 a.m.",
    "The $500 billion drug industry is stumbling badly",
    "Classic video games like Frogger and Pong",
    "US",
    "Sean O #39;Keefe",
    "Tokyo",
    "Michael Phelps",
    "Provides online listings of apartment and house rentals",
    "Sporadic gunfire and shelling took place, resulting in injuries to seven Georgian servicemen",
    "40 days"
    ]




In [ ]:
q_embeds = embedder.get_embeddings(questions)

In [ ]:
# ожидаемое максимальное кол-во результатов поиска до 1000
results_cos = collection_cos.query(q_embeds,1000)
print('Base news ',  news[0])
print('Result\n ', results_cos['documents'][0][:10])

In [ ]:
results_l2 = collection_l2.query(q_embeds,1000)
print('Base news ',  news[0])
print('Result\n ', results_l2['documents'][0][:10])

In [ ]:
results_ip = collection_Ip.query(q_embeds,1000)
print('Base news ',  news[0])
print('Result\n ', results_ip['documents'][0][:10])

## Лабораторная работа №6 (Question Answering)

Необходимо запустить и протестировать QA на основе LLM модели, можно выбрать любую LLM модель (рекомендуется искать на huggingface):


> на основе llama.cpp - можно запустить на GPU и/или на CPU, примеры:



In [ ]:
# инициализация вопросно-ответной модели roberta-base-squad2
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
model_name = "deepset/roberta-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

In [ ]:
q_embeds = embedder.get_embeddings(questions)
results = collection_cos.query(q_embeds,5)
print('Base news ',  news[0])
print('Student answer ',  answers[0])
print('Result\n ', results['documents'][0])


In [ ]:
!pip install evaluate
!pip install bert_score
from evaluate import load
bertscore = load("bertscore")

In [ ]:
bs_all = []
for q, a, index in zip(questions, answers, range(len(answers))):
  QA_input = {'question': q,
             'context': ' '.join(results['documents'][index])}
  res = nlp(QA_input)
  bs = bertscore.compute(predictions=[res['answer']], references=[a], lang="en")
  bs_all.append(bs)

  print(f'Question: {q}\nAnswer: {res["answer"]}\nUser answer: {a}\nScore: {bs["f1"][0]}\n ')

In [ ]:
# среднее значение метрики по всем вопросам
f1_scores = [bs['f1'][0] for bs in bs_all]
sum(f1_scores)/len(f1_scores)

# **Gradio**

In [ ]:
!pip install gradio==3.48.0

In [ ]:
!pip install kaleido

In [ ]:
import gradio as gr

In [ ]:
#QA call
def echo(question, history):
    q_embeds = embedder.get_embeddings([question])
    results = collection_cos.query(q_embeds,5)
    QA_input = {'question': question,
             'context': ' '.join(results['documents'][0])}
    res = nlp(QA_input)
    return res['answer']

In [ ]:
# графический интерфейс
demo = gr.ChatInterface(fn=echo, examples=["hello", "hola", "merhaba"], title="QA Bot")
demo.launch()